In [1]:
import ROOT
from root_numpy import root2array, tree2array, root2rec, testdata
import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize # MINIMIZATION
import copy

Welcome to JupyROOT 6.12/06


## Prepare data structure and functions for single tracking

### Line

In [2]:
class Line:
    def __init__(self, x=0, y=0, z=0, dx=0, dy=0, dz=0, point=[], direction=[], params=[], det_id=0):
        if len(params) == 6:
            self.x,  self.y,  self.z  = params[:3] 
            self.dx, self.dy, self.dz = params[3:]
        elif len(point) == 3 and len(direction) == 3:
            self.x,  self.y,  self.z  = point
            self.dx, self.dy, self.dz = direction
        else:
            self.x,  self.y,  self.z  =  x,  y,  z
            self.dx, self.dy, self.dz = dx, dy, dz
        
        # just to make this line a detector Line
        self.det_id = int(det_id)

    def __repr__(self):
        return "Line: det = {}\t p = ({:.3f}, {:.3f}, {:.3f})\t v = [{:.3f}, {:.3f}, {:.3f}]".format(
            self.det_id, self.x, self.y, self.z, self.dx, self.dy, self.dz
        )
            
    def distance(self, other):
        n_vector = np.cross(self.line_vector(), other.line_vector())
        s_o_vector = np.array([self.x - other.x, self.y - other.y, self.z - other.z])
        return np.linalg.norm( np.dot(n_vector, s_o_vector) ) / np.linalg.norm(n_vector)

    def line_vector(self):
        return np.array([self.dx, self.dy, self.dz])

### Open root file

In [3]:
geometry_root = root2array('det_geometry.root', 'T')
reco_flat = root2array('reco_0_flat.root', 'T')

### Prepare dataframes

In [4]:
geom_df = pd.DataFrame(geometry_root) 
df_all  = pd.DataFrame(reco_flat)

single_df = df_all.loc[(df_all['uLineSize'] == 1) & (df_all['vLineSize'] == 1)]     # OLNY RPS WITH SINGLE TRACKING
single_groups_df = single_df[['eventID', 'groupID', 'rpID']]\
    .drop_duplicates()\
    .groupby(['eventID', 'groupID'])\
    .size()\
    .reset_index(name='counts')
single_groups_df = single_groups_df.loc[(single_groups_df['counts'] == 3)]          # ONLY HITS WHERE ALL RPS 
                                                                                    # FROM GROUP WAS HIT

print("all tracks shape: {}".format(df_all.shape))
print("single tracking shape: {}".format(single_df.shape))
print("single_groups shape: {}".format(single_groups_df.shape))

all tracks shape: (5542, 10)
single tracking shape: (5097, 10)
single_groups shape: (156, 3)


### Hit lines from dataframes
Needed: geometry of detectors and hit positions

In [5]:
class Direction:
    '''
    Wektor jednostkowy
    '''
    def __init__(self, dx=1.0, dy=0.0):
        self.dx = dx
        self.dy = dy
    
    def __repr__(self):
        return "dx = {}\tdy={}".format(self.dx, self.dy)

    def get_perpendicular(self):
        return Direction(self.dy, -self.dx)
        
    
class Vector2D:
    def __init__(self, x=0.0, y=0.0, start=None, end=None):
        if start is None or end is None:
            self.x = x
            self.y = y
        else:
            self.x = end.x - start.x
            self.y = end.y - start.y
                 
    def __repr__(self):
        return "x = {}\ty = {}".format(self.x, self.y)
    

class Point2D:
    def __init__(self, x=0.0, y=0.0):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return "x = {}\ty = {}".format(self.x, self.y)
    
    def move(self, vector):
        self.x += vector.x
        self.y += vector.y
    
    def rotate_around_point(self, other, direction):
        s = direction.dy
        c = direction.dx
        
        new_x = c * (self.x - other.x) - s * (self.y - other.y) + other.x
        new_y = s * (self.x - other.x) + c * (self.y - other.y) + other.y
    
        self.x = new_x
        self.y = new_y

        
class RPSilicon:
    
    RP_Det_Fid_Top_a   = 0.301  # [mm]
    RP_Det_Fid_Top_b   = 1.0275 # [mm]
    RP_Det_Fid_Left_a  = 0.301  # [mm]
    RP_Det_Fid_Left_c  = 0.2985 # [mm]
    RP_Det_Fid_Right_b = 1.0275 # [mm]
    RP_Det_Fid_Right_d = 1.0255 # [mm]

    RP_Det_dx_0 = ( RP_Det_Fid_Left_c  -  RP_Det_Fid_Right_b) / 2.0 # [mm]
    RP_Det_dy_0 = ( RP_Det_Fid_Right_d  -  RP_Det_Fid_Left_a) / 2.0
    
    def __init__(self, center, direction, siId):
        self.center = center
        self.direction = direction
        self.siId = siId
        
        fid_center = copy.deepcopy(self.center)
        fid_center.move(Vector2D(RPSilicon.RP_Det_dx_0, RPSilicon.RP_Det_dy_0))
        fid_center.rotate_around_point(self.center, self.get_rot_direction())
        
        self.fid_center = fid_center
    
    def get_rot_direction(self):
        arm = int(self.siId / 1000) 
        if   arm == 0 and self.siId % 2 == 0:  
            return self.direction
        elif arm == 0 and self.siId % 2 == 1:
            return Direction(self.direction.dy, -self.direction.dx)
        elif arm == 1 and self.siId % 2 == 0:
            return Direction(self.direction.dy, -self.direction.dx)
        else:
            return self.direction

In [6]:
DET_Z_TRANSLATION = 0 # SET WITHIN EVENT PROCESSING


def get_hit_lines(single_hits_df, geom_df):
    '''
        reco_0_flat.root:     recoID  eventID  armID  groupID  rpID  uLineSize  vLineSize  siliconID   position  sigma_
        det_geometry.root:    detId   x        y      z        dx    dy
    '''
    
    hit_lines = []

    for h_index, h_row in single_hits_df.iterrows():
        
        h_position = h_row['position']
        det_ID = 10 * h_row['rpID'] + h_row['siliconID']        
        det_info = geom_df.loc[(geom_df['detId'] == det_ID)].iloc[0]
    
        rp_silicon = RPSilicon(Point2D(det_info['x'], det_info['y']), Direction(dx=det_info['dx'], dy=det_info['dy']), det_ID)
        
        x = rp_silicon.fid_center.x + h_position * det_info['dx']    # was: det_info['x'], but now: fiducial included [mm]
        y = rp_silicon.fid_center.y + h_position * det_info['dy']    # was: det_info['y'], but now: fiducial included [mm] 
        
        z = det_info['z'] * 1000 + DET_Z_TRANSLATION     # z [mm] with translation to operate on smaller numbers
        dx = - det_info['dy']  
        dy = det_info['dx'] 
        dz = 0
        
        new_line = Line(x, y, z, dx, dy, dz, det_id=det_ID)
        hit_lines.append(new_line)
        
    return hit_lines

### Printing fitting solution

In [7]:
class Hit:
    def __init__(self, x=0, y=0, z=0):
        self.x = x
        self.y = y
        self.z = z


def print_pretty_solution(solution, geom_df):
    print("FULL SOLUTION: \n{}".format(solution))
    print("Sum of distances: {}".format(solution.fun))
    print("Solution: [\n\tx = {}\n\ty = {} \n\tz = {} \n\tdx = {} \n\tdy = {} \n\tdz = {} ]".format(
        solution.x[0],
        solution.x[1],
        solution.x[2], # - DET_Z_TRANSLATION,
        solution.x[3],
        solution.x[4],
        solution.x[5])
    )
    print("]")
    print("\nDistances from detector lines and hit position")
    print("ID\tDistance from hit line [mm]\t x [mm]\t\ty [mm]\t\tz [m]")

    plane_hit = Hit()

    for line in LINE_SET:
        sol_x, sol_y, sol_z = solution.x[0], solution.x[1], solution.x[2]
        sol_dx, sol_dy, sol_dz = solution.x[3], solution.x[4], solution.x[5]
        
        det_info = geom_df.loc[(geom_df['detId'] == line.det_id)].iloc[0]
        det_z = det_info['z']
        k = (det_z - (sol_z - DET_Z_TRANSLATION)) / sol_dz
        
        plane_hit.x = sol_x + k * sol_dx
        plane_hit.y = sol_y + k * sol_dy
        plane_hit.z = det_z

        print("{}\t{:.5f}\t\t\t\t({:.5f}\t{:.5f}\t{})".format(line.det_id,
                                                              Line(params=solution.x).distance(line),
                                                              plane_hit.x,
                                                              plane_hit.y,
                                                              plane_hit.z))
    print("\n")

### Minimization for each rp group where is single-tracking 

In [8]:
k = [17.603449563, 22.1827574193, 27.22961824, 24.1261802529, 22.1439723535]
s = set(k)
d = {} 

for entry in s:
    d[entry] = 0

for entry in k:
    d[entry] = d[entry] + 1

In [10]:
# REQUIRED BY MY CHI2 FUNCTION
LINE_SET = []

# OBJECTIVE FUNCTION
def chi2(params):
    # TO DO: make it real chi2
    line = Line(params=params)                                  # CREATING FITTED LINE FROM PARAMS
    return np.sum([line.distance(other) for other in LINE_SET]) # SUM OF DISTANCES



# SEED FOR MINIMIZING
def get_x0(first_hit_info, geom_df):    
    det_id   = first_hit_info['rpID'] * 10 + first_hit_info['siliconID'] 
    det_info = geom_df.loc[(geom_df['detId'] == det_id)].iloc[0]
    
    det_x = 0.0 # det_info['x']
    det_y = 0.0 # det_info['y']
    det_z = det_info['z'] * 1000 + DET_Z_TRANSLATION 
    
    return [det_x, det_y, det_z, 0.1, 0.1, np.sign(det_z)]

def get_constraints():
    def constraint1(params):
        return 1.0 - np.sum([dir**2 for dir in params[3:]]) # dx^2 + dy^2 + dz^2 = 1

    con1 = {'type': 'eq', 'fun':constraint1}
    return [con1]


def get_bounds():
    # BOUNDS FOR PARAMETERS
    b_x   = (   -50.0,    50.0)
    b_y   = (   -50.0,    50.0)
    b_z   = (-20000.0, 20000.0)
    b_dir = (    -1.0,     1.0)
    return (b_x, b_y, b_z, b_dir, b_dir, b_dir)


# FOR EACH EVENT AND GROUP CALCULATE LINE
row_number = 0
glob_max_hit_dist = 0.0
true_max_distance = 0.0
false_max_distance = 0.0

rp_groups = {
    1 : [4, 20, 24],
    2 : [5, 21, 25],
    3 : [3, 22, 23],
    4 : [104, 120, 124],
    5 : [105, 121, 125],
    6 : [103, 122, 123]
}


interesting_rows =[9, 15, 18, 22, 26, 29,
31, 32, 34, 35, 40, 43, 47, 50, 55, 56,
57, 62, 64, 65, 66, 68, 69, 70, 75, 80, 82,
89, 95, 97, 98, 102, 104, 109, 111, 112, 115, 121,
122, 132, 137, 138, 142, 144, 145, 147, 150, 152, 153]

for index, row in single_groups_df.iterrows():
    
    # GETTING HITS FOR SPECIFIED GROUP AND EVENT
    eventID, groupID = row['eventID'], row['groupID']
    single_hits_df = single_df.loc[(single_df['eventID'] == eventID) & (single_df['groupID'] == groupID)]
    
    # APPLYING Z TRANSLATION --> WE DO NOT WANT SUCH BIG NUMBERS (|Z| ~ 210 000 [mm])
    if groupID > 3:
        DET_Z_TRANSLATION = -210000
    else:
        DET_Z_TRANSLATION = 210000

    LINE_SET = get_hit_lines(single_hits_df, geom_df)
    
    # SEED SOLUTION - CENTER OF FIRST PLANE [x, y, z] AND DIRECTION ALONG THE BEAM [0, 0, [+|-] 1] 
    x0 = get_x0(single_hits_df.iloc[0], geom_df)    
    
    # RUNNING OPTIMIZING METHODS TO GET SOLUTION
    METHODS = ['Powell', 'Nelder-Mead']
    
    method = 'SLSQP'
    solution = minimize(chi2, x0, method=method, constraints=get_constraints(), bounds=get_bounds())
    
    if solution.success != True:
        nelder_solution = minimize(chi2, x0, method='Nelder-Mead', constraints=get_constraints(), bounds=get_bounds())
        
        solution = nelder_solution
        method = 'Nelder-Mead'
        
        if nelder_solution.fun < 1.0:
            method = 'Nelder'
            solution = nelder_solution
        else:
            powell_solution = minimize(chi2, x0, method='Powell', constraints=get_constraints(), bounds=get_bounds())
            
            if powell_solution.fun < nelder_solution:
                method = 'Powell'
                solution = powell_solution
            else:
                method = 'Nelder'
                solution = nelder_solution
            
    
    if solution.success == True:
        true_max_distance = np.maximum(true_max_distance, solution.fun)
    else:
        false_max_distance = np.maximum(false_max_distance, solution.fun)
                
    
    max_hit_dist = np.amax([ Line(params=solution.x).distance(line) for line in LINE_SET])
    glob_max_hit_dist = np.maximum(glob_max_hit_dist, max_hit_dist)
    
    print("Row {} Event: {} Group: {} Method: {}\tDistance sum: {:.5f}\tMost Distant Hit: {:.5f}\tSuccess: {}"
              .format(row_number, eventID, groupID, method, solution.fun, max_hit_dist, solution.success))
    
    
#     print_pretty_solution(solution, geom_df)   
#     LINE_SET = get_hit_lines(single_hits_df, geom_df)
    
#     for rp_id in rp_groups[groupID]:
#         single_rp_hits_df = single_df.loc[(single_df['eventID'] == eventID) & 
#                                           (single_df['groupID'] == groupID) &
#                                           (single_df['rpID'] == rp_id)]
    
#         LINE_SET = get_hit_lines(single_rp_hits_df, geom_df)    
        
#         x0 = get_x0(single_rp_hits_df.iloc[0], geom_df)    
#         solution = minimize(chi2, x0, method='SLSQP', constraints=get_constraints(), bounds=get_bounds())
#         max_distance = np.maximum(max_distance, solution.fun)

        
#         print("Row number {}\tGroup: {}\tRP: {}\tDistances sum: {:.5f}\tSuccess: {}\tMax distance: {}"
#                   .format(row_number, groupID, rp_id, solution.fun, solution.success, max_distance))
        
    row_number = row_number + 1
    

print("THE END")


Row 0 Event: 2 Group: 2 Method: Nelder	Distance sum: 0.42049	Most Distant Hit: 0.04462	Success: True
Row 1 Event: 2 Group: 4 Method: SLSQP	Distance sum: 0.53292	Most Distant Hit: 0.04629	Success: True
Row 2 Event: 3 Group: 1 Method: SLSQP	Distance sum: 0.50122	Most Distant Hit: 0.06738	Success: True
Row 3 Event: 3 Group: 2 Method: SLSQP	Distance sum: 0.69759	Most Distant Hit: 0.08579	Success: True
Row 4 Event: 3 Group: 4 Method: SLSQP	Distance sum: 0.49803	Most Distant Hit: 0.05562	Success: True
Row 5 Event: 4 Group: 2 Method: SLSQP	Distance sum: 0.71539	Most Distant Hit: 0.05798	Success: True
Row 6 Event: 5 Group: 1 Method: Nelder	Distance sum: 0.78325	Most Distant Hit: 0.10546	Success: True
Row 7 Event: 5 Group: 5 Method: SLSQP	Distance sum: 0.58450	Most Distant Hit: 0.05553	Success: True
Row 8 Event: 6 Group: 1 Method: SLSQP	Distance sum: 0.55677	Most Distant Hit: 0.05350	Success: True
Row 9 Event: 6 Group: 5 Method: Powell	Distance sum: 7.61974	Most Distant Hit: 0.67707	Success: Tr

Row 81 Event: 53 Group: 5 Method: Nelder	Distance sum: 0.82346	Most Distant Hit: 0.09543	Success: True
Row 82 Event: 54 Group: 2 Method: Powell	Distance sum: 8.96236	Most Distant Hit: 0.72236	Success: True
Row 83 Event: 54 Group: 4 Method: SLSQP	Distance sum: 0.50696	Most Distant Hit: 0.04789	Success: True
Row 84 Event: 55 Group: 1 Method: SLSQP	Distance sum: 0.37498	Most Distant Hit: 0.04841	Success: True
Row 85 Event: 55 Group: 5 Method: SLSQP	Distance sum: 0.50793	Most Distant Hit: 0.05586	Success: True
Row 86 Event: 56 Group: 2 Method: Nelder	Distance sum: 0.63691	Most Distant Hit: 0.04957	Success: True
Row 87 Event: 56 Group: 4 Method: SLSQP	Distance sum: 0.52815	Most Distant Hit: 0.05742	Success: True
Row 88 Event: 57 Group: 1 Method: SLSQP	Distance sum: 0.58542	Most Distant Hit: 0.07687	Success: True
Row 89 Event: 57 Group: 5 Method: Powell	Distance sum: 10.01421	Most Distant Hit: 0.87120	Success: True
Row 90 Event: 58 Group: 1 Method: SLSQP	Distance sum: 0.55090	Most Distant Hi